Many entity search failed because we couldn't locate query characters within entity's summary page. This time we'll fetch he entity descriptions, and see if that'd help.

In [36]:
import unicodecsv as csv
import urllib
import logging
import threading
import Queue
import time
from bs4 import BeautifulSoup
import tqdm
baike_file = '../entities_db/baike.csv'

ImportError: No module named tqdm

In [7]:
link_data = dict((line['entity_name'], line['link'])for line in csv.DictReader(open(baike_file)) if line['link'])

In [13]:
print len(link_data)
sample_link = link_data.values()[0]

95646


In [35]:
def FetchLink(link):
    n_retry = 0
    while n_retry <= 3:
        try:
            page = urllib.urlopen(sample_link).read()
            soup = BeautifulSoup(page)
            text = "\n".join([i.text for i in soup('div', {'class' : 'para'})])
            return text
        except Exception, e:
            logging.error("Error retrieving %s", link)
            n_retry += 1
            time.sleep(3)
    logging.error("Gave up %s", link)
    return None

print sample_link
print FetchLink(sample_link)

http://baike.baidu.com/view/109039.htm
吴浩康，1983年6月13日出生于香港，歌手。
2002年参加无线电视自办的全球华人新秀赛总选获季军，开始出唱片、拍广告、电视剧和电影，曾因藏毒、殴打等控罪惹上官非。现属于香港英皇娱乐集团。
2012年，他与吉他手Ryan Lam、低音吉他手Johnny Choi及鼓手Masaki Heung组成Closer乐队，并于6月15日推出唱片《Get Closer》。
吴浩康从祖父起，三代均生于香港偏郊的茶果岭，两岁时举家搬往元朗，父吴志安则仍在茶果岭经营权记搬机运输公司，母亲为刘柳薇，家中共有3名姐姐，排行最小的也大他6年。
1989年，其家人以99万元购入元朗锦锈花园一物业，小学时就读于元朗佛教荣茵学校，中学时就读于元朗伯裘书院。
11岁时偷学泰拳，曾赢得奖杯，中四已辍学，曾教过两年guitar。





生活照
(11张)






2002年参加全球华人新秀赛总选获季军，被英皇娱乐招揽，初出道时曾被冠以“英俊版陈奕迅的称号。
两年内推出过两张 EP《Deep》及《Deep In Music》，《Deep：Inside》则是首张正式大碟，其中“孩子王”等曾在多个流行榜获得冠军。其公开女友为无线电视艺人胡定欣（已于2006年8月和平分手）；另与一新进歌手林子善是中学同学。
感情生活
2006年，吴浩康与庄思敏公开拍拖。[1] 

2011年，庄思敏单方面向传媒承认分手，其后吴浩康表示有意复合，却遭拒绝。[1] 






出席活动
(21张)






英皇三周年慈善演唱会 （红磡香港体育馆07/12/2002）加州红903十一团火音乐会（九龙湾国际展贸中心 11/2004）
曾担任大使列表
东华三院委任为“东华三院学校健康大使”
香港伤残青年协会委任为“关怀大使”
香港武术联会委任为“武术推广大使”
2002年全球华人新秀歌唱大赛香港区选拔赛~ 金咪大奖
2003年 PM第二届夏日人气颁奖典礼 ~ PM夏日靓声男新人
香港电台《第26届十大中文金曲颁奖礼》最有前途新人奖 （男歌手）（银奖）
2004年商业电台《03年度叱吒乐坛流行榜颁奖礼》叱吒乐坛新力军男歌手 （银奖）
加拿大中文电台《加拿大至Hit中文歌曲排行榜 - 03年度全国乐迷投票》最受欢迎男新人（粤）（银奖）
无线电视《

In [ ]:
task_queue = Queue.Queue()
write_queue = Queue.Queue()

class Worker(threading.Thread):
    def run(self):
        while True:
            name, link = task_queue.get()
            content = FetchLink(link)
            write_queue.put((name, content))
            
class Writer(threading.Thread):
    def __init__(self, outputfile, total_n):
        threading.Thread.__init__(self)
        self.outfile = open(outputfile, 'w')
        self.writer = csv.DictWriter(self.outfile, ['entity_name', 'content'])
        self.writer.writeheader()
    
    def run(self):
        for i in tqdm(range(total_n)):
            name, content = write_queue.get()
            self.writer.writerow({'entity_name' : name, 'content' : content})